In [ ]:
from notebook_prelude import *

In [ ]:
import re

LOG_DIR = 'tmp/logs'
logs = sorted(glob('{}/*.log'.format(LOG_DIR)))
log = logs[-1]

def get_log(log):
    with open(log) as f:
        return [x for x in f.read().split('\n')]

def get_finish_times(log_lines):
    FINISHED_REGEXP = r'\d: (.+?) +- (.+?) +- Finished \(time=(.+?)\)'
    finished_lines = [x.strip() for x in log_lines if x.count('(time=') == 1]
    finish_times = re.findall(FINISHED_REGEXP, '\n'.join(finished_lines))
    return finish_times


df = pd.DataFrame(get_finish_times(get_log(log)), columns = ['type', 'name', 'time'])

In [ ]:
def time_str_2_seconds(x):
    parts = x.split(':')
    seconds = 0
    for i, part in enumerate(reversed(parts)):
        seconds += (np.power(60, i)) * int(part)
    return seconds

df['dataset'] = df.name.apply(filename_utils.get_dataset_from_filename)
df['time_as_seconds'] = df.time.apply(time_str_2_seconds)
df['time_as_minutes'] = df.time_as_seconds.apply(lambda x: int(x / 60))

In [ ]:
df.groupby(['dataset', 'type']).time_as_minutes.max().to_frame()